In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Mediapipe setup
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=5,  # Support up to 5 faces
    refine_landmarks=True
)
drawing_utils = mp.solutions.drawing_utils
LIPS = mp_face_mesh.FACEMESH_LIPS
LEFT_IRIS = [468, 469, 470, 471]
RIGHT_IRIS = [473, 474, 475, 476]

# Emotion rules
emotions = {
    "happy": {"emoji": "😊", "color": (0, 255, 0)},
    "sad": {"emoji": "😢", "color": (255, 0, 0)},
    "angry": {"emoji": "😠", "color": (0, 0, 255)},
    "surprise": {"emoji": "😲", "color": (0, 255, 255)},
    "neutral": {"emoji": "😐", "color": (255, 255, 255)},
    "fear": {"emoji": "😨", "color": (255, 140, 0)},
    "disgust": {"emoji": "🤢", "color": (138, 43, 226)}
}

def distance(p1, p2):
    return np.linalg.norm(np.array([p1.x, p1.y]) - np.array([p2.x, p2.y]))

def get_emotion(landmarks):
    top_lip = landmarks[13]
    bottom_lip = landmarks[14]
    left_mouth = landmarks[61]
    right_mouth = landmarks[291]
    left_eye_top = landmarks[159]
    left_eye_bottom = landmarks[145]
    right_eye_top = landmarks[386]
    right_eye_bottom = landmarks[374]
    iris_left = landmarks[468]

    face_width = distance(landmarks[234], landmarks[454])
    mouth_open = distance(top_lip, bottom_lip) / face_width
    mouth_stretch = distance(left_mouth, right_mouth) / face_width
    eye_open = (distance(left_eye_top, left_eye_bottom) + distance(right_eye_top, right_eye_bottom)) / (2 * face_width)
    eye_center_y = (left_eye_top.y + left_eye_bottom.y + right_eye_top.y + right_eye_bottom.y) / 4
    sad_offset = iris_left.y - eye_center_y

    if mouth_stretch > 0.40 and mouth_open < 0.06:
        return "happy"
    elif mouth_open >= 0.12:
        return "surprise"
    elif 0.06 < mouth_open < 0.12:
        return "fear"
    elif sad_offset > 0.01 and eye_open < 0.04:
        return "sad"
    elif mouth_open < 0.03 and eye_open < 0.08 and mouth_stretch < 0.38:
        return "disgust"
    elif eye_open > 0.096 and mouth_open < 0.06:
        return "angry"
    else:
        return "neutral"

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)
    avatar_canvas = np.zeros_like(frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmark_list = face_landmarks.landmark
            emotion = get_emotion(landmark_list)
            color = emotions[emotion]["color"]
            emoji = emotions[emotion]["emoji"]

            # Draw mesh
            drawing_utils.draw_landmarks(
                avatar_canvas, face_landmarks,
                mp_face_mesh.FACEMESH_TESSELATION, None,
                drawing_utils.DrawingSpec(color=color, thickness=1, circle_radius=1)
            )
            drawing_utils.draw_landmarks(
                avatar_canvas, face_landmarks, LIPS, None,
                drawing_utils.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1)
            )

            # Draw irises
            for idx in LEFT_IRIS + RIGHT_IRIS:
                pt = landmark_list[idx]
                cx, cy = int(pt.x * w), int(pt.y * h)
                cv2.circle(avatar_canvas, (cx, cy), 2, (0, 255, 255), -1)

            # Show emotion
            cx, cy = int(landmark_list[0].x * w), int(landmark_list[0].y * h)
            cv2.putText(avatar_canvas, f"{emotion.upper()} {emoji}", (cx - 50, cy - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    # Show windows
    cv2.imshow("Webcam Feed", cv2.resize(frame, (640, 480)))
    cv2.imshow("Avatar Emotion Mesh", cv2.resize(avatar_canvas, (640, 480)))

    if cv2.waitKey(1) & 0xFF == 27:  # ESC to exit
        break

cap.release()
cv2.destroyAllWindows()


In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Mediapipe setup
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=5,
    refine_landmarks=True
)
drawing_utils = mp.solutions.drawing_utils
LIPS = mp_face_mesh.FACEMESH_LIPS
LEFT_IRIS = [468, 469, 470, 471]
RIGHT_IRIS = [473, 474, 475, 476]

# Emotion configuration
emotions = {
    "happy": {"emoji": "😊", "color": (0, 255, 0)},
    "sad": {"emoji": "😢", "color": (255, 0, 0)},
    "angry": {"emoji": "😠", "color": (0, 0, 255)},
    "surprise": {"emoji": "😲", "color": (0, 255, 255)},
    "neutral": {"emoji": "😐", "color": (255, 255, 255)},
    "fear": {"emoji": "😨", "color": (255, 140, 0)},
    "disgust": {"emoji": "🤢", "color": (138, 43, 226)}
}

def distance(p1, p2):
    return np.linalg.norm(np.array([p1.x, p1.y]) - np.array([p2.x, p2.y]))

def get_emotion(landmarks):
    top_lip = landmarks[13]
    bottom_lip = landmarks[14]
    left_mouth = landmarks[61]
    right_mouth = landmarks[291]
    left_eye_top = landmarks[159]
    left_eye_bottom = landmarks[145]
    right_eye_top = landmarks[386]
    right_eye_bottom = landmarks[374]
    iris_left = landmarks[468]

    face_width = distance(landmarks[234], landmarks[454])
    mouth_open = distance(top_lip, bottom_lip) / face_width
    mouth_stretch = distance(left_mouth, right_mouth) / face_width
    eye_open = (distance(left_eye_top, left_eye_bottom) + distance(right_eye_top, right_eye_bottom)) / (2 * face_width)
    eye_center_y = (left_eye_top.y + left_eye_bottom.y + right_eye_top.y + right_eye_bottom.y) / 4
    sad_offset = iris_left.y - eye_center_y

    if mouth_stretch > 0.40 and mouth_open < 0.06:
        return "happy"
    elif mouth_open >= 0.12:
        return "surprise"
    elif 0.06 < mouth_open < 0.12:
        return "fear"
    elif sad_offset > 0.01 and eye_open < 0.04:
        return "sad"
    elif mouth_open < 0.03 and eye_open < 0.08 and mouth_stretch < 0.38:
        return "disgust"
    elif eye_open > 0.096 and mouth_open < 0.06:
        return "angry"
    else:
        return "neutral"

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    avatar_canvas = np.zeros_like(frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmark_list = face_landmarks.landmark
            emotion = get_emotion(landmark_list)
            color = emotions[emotion]["color"]
            emoji = emotions[emotion]["emoji"]

            # Draw avatar mesh
            drawing_utils.draw_landmarks(
                avatar_canvas, face_landmarks,
                mp_face_mesh.FACEMESH_TESSELATION, None,
                drawing_utils.DrawingSpec(color=color, thickness=1, circle_radius=1)
            )
            drawing_utils.draw_landmarks(
                avatar_canvas, face_landmarks, LIPS, None,
                drawing_utils.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1)
            )

            # Draw irises
            for idx in LEFT_IRIS + RIGHT_IRIS:
                pt = landmark_list[idx]
                cx, cy = int(pt.x * w), int(pt.y * h)
                cv2.circle(avatar_canvas, (cx, cy), 2, (0, 255, 255), -1)

            # Emotion label on avatar face
            cx, cy = int(landmark_list[1].x * w), int(landmark_list[1].y * h)
            cv2.putText(avatar_canvas, f"{emotion.upper()} {emoji}", (cx - 50, cy - 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

            # Also overlay emotion text on real webcam frame
            cv2.putText(frame, f"{emotion.upper()} {emoji}", (cx - 50, cy - 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    # Resize both views to same size
    frame_resized = cv2.resize(frame, (480, 360))
    avatar_resized = cv2.resize(avatar_canvas, (480, 360))

    # Combine side-by-side
    combined_view = np.hstack((frame_resized, avatar_resized))
    cv2.imshow("🤳 Real Face + Avatar 🤖", combined_view)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC key to exit
        break

cap.release()
cv2.destroyAllWindows()


In [3]:
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3

# Setup speech engine
engine = pyttsx3.init()
engine.setProperty('rate', 160)

# Setup Mediapipe
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=5, refine_landmarks=True)
drawing_utils = mp.solutions.drawing_utils

# Face mesh constants
LIPS = mp_face_mesh.FACEMESH_LIPS
LEFT_IRIS = [468, 469, 470, 471]
RIGHT_IRIS = [473, 474, 475, 476]

# Emotion config
emotions = {
    "happy": {"emoji": "😊", "color": (0, 255, 0), "msg": "You look happy!"},
    "sad": {"emoji": "😢", "color": (255, 0, 0), "msg": "You seem sad."},
    "angry": {"emoji": "😠", "color": (0, 0, 255), "msg": "You look angry!"},
    "surprise": {"emoji": "😲", "color": (0, 255, 255), "msg": "You're surprised!"},
    "neutral": {"emoji": "😐", "color": (200, 200, 200), "msg": "You look neutral."},
    "fear": {"emoji": "😨", "color": (255, 140, 0), "msg": "You look scared."},
    "disgust": {"emoji": "🤢", "color": (138, 43, 226), "msg": "You're feeling disgusted."}
}

prev_emotion = ""

def distance(p1, p2):
    return np.linalg.norm(np.array([p1.x, p1.y]) - np.array([p2.x, p2.y]))

def get_emotion(landmarks):
    top_lip = landmarks[13]
    bottom_lip = landmarks[14]
    left_mouth = landmarks[61]
    right_mouth = landmarks[291]
    left_eye_top = landmarks[159]
    left_eye_bottom = landmarks[145]
    right_eye_top = landmarks[386]
    right_eye_bottom = landmarks[374]
    iris_left = landmarks[468]

    face_width = distance(landmarks[234], landmarks[454])
    mouth_open = distance(top_lip, bottom_lip) / face_width
    mouth_stretch = distance(left_mouth, right_mouth) / face_width
    eye_open = (distance(left_eye_top, left_eye_bottom) + distance(right_eye_top, right_eye_bottom)) / (2 * face_width)

    eye_center_y = (left_eye_top.y + left_eye_bottom.y + right_eye_top.y + right_eye_bottom.y) / 4
    sad_offset = iris_left.y - eye_center_y

    if mouth_stretch > 0.40 and mouth_open < 0.06:
        return "happy"
    elif mouth_open >= 0.12:
        return "surprise"
    elif 0.06 < mouth_open < 0.12:
        return "fear"
    elif sad_offset > 0.01 and eye_open < 0.04:
        return "sad"
    elif mouth_open < 0.03 and eye_open < 0.08 and mouth_stretch < 0.38:
        return "disgust"
    elif eye_open > 0.096 and mouth_open < 0.06:
        return "angry"
    else:
        return "neutral"

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    avatar_canvas = np.zeros_like(frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmark_list = face_landmarks.landmark
            emotion = get_emotion(landmark_list)
            color = emotions[emotion]["color"]
            emoji = emotions[emotion]["emoji"]

            # Draw avatar face only (not on webcam feed)
            drawing_utils.draw_landmarks(
                avatar_canvas, face_landmarks,
                mp_face_mesh.FACEMESH_TESSELATION, None,
                drawing_utils.DrawingSpec(color=color, thickness=1, circle_radius=1)
            )
            drawing_utils.draw_landmarks(
                avatar_canvas, face_landmarks, LIPS, None,
                drawing_utils.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1)
            )

            for idx in LEFT_IRIS + RIGHT_IRIS:
                pt = landmark_list[idx]
                cx, cy = int(pt.x * w), int(pt.y * h)
                cv2.circle(avatar_canvas, (cx, cy), 2, (0, 255, 255), -1)

            # Show emotion label in avatar
            cx, cy = int(landmark_list[0].x * w), int(landmark_list[0].y * h)
            cv2.putText(avatar_canvas, f"{emotion.upper()} {emoji}", (cx - 50, cy - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

            # Trigger speech only when emotion changes
            if emotion != prev_emotion:
                engine.say(emotions[emotion]["msg"])
                engine.runAndWait()
                prev_emotion = emotion

    # Show combined frame: webcam left, avatar right
    combined_view = np.hstack((frame, avatar_canvas))
    cv2.imshow("Real Face | Avatar Emotion", cv2.resize(combined_view, (1280, 480)))

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [4]:
def draw_cartoon_avatar(canvas, emotion, cx, cy):
    cv2.circle(canvas, (cx, cy), 50, (255, 255, 255), -1)  # head

    # Eyes
    cv2.circle(canvas, (cx - 15, cy - 15), 5, (0, 0, 0), -1)
    cv2.circle(canvas, (cx + 15, cy - 15), 5, (0, 0, 0), -1)

    # Mouth expressions
    if emotion == "happy":
        cv2.ellipse(canvas, (cx, cy + 10), (15, 10), 0, 0, 180, (0, 255, 0), 2)
    elif emotion == "sad":
        cv2.ellipse(canvas, (cx, cy + 20), (15, 10), 0, 180, 360, (255, 0, 0), 2)
    elif emotion == "angry":
        cv2.line(canvas, (cx - 15, cy + 15), (cx + 15, cy + 15), (0, 0, 255), 2)
    else:
        cv2.line(canvas, (cx - 10, cy + 15), (cx + 10, cy + 15), (200, 200, 200), 2)


In [5]:
import math
import time

def get_avatar_offset():
    t = time.time()
    offset = int(10 * math.sin(t * 3))  # Bouncing effect
    return offset


In [6]:
cx, cy = 320, 240  # Avatar center
cy += get_avatar_offset()
draw_cartoon_avatar(avatar_canvas, emotion, cx, cy)


In [7]:
import pyttsx3
engine = pyttsx3.init()

last_spoken_emotion = None

if emotion != last_spoken_emotion:
    engine.say(f"You look {emotion}")
    engine.runAndWait()
    last_spoken_emotion = emotion


In [13]:
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3
import threading
import time

# Initialize TTS engine
engine = pyttsx3.init()
engine.setProperty('rate', 160)

# Mediapipe face mesh setup
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)
drawing_utils = mp.solutions.drawing_utils

# Facial landmark sets
LIPS = mp_face_mesh.FACEMESH_LIPS
LEFT_IRIS = [468, 469, 470, 471]
RIGHT_IRIS = [473, 474, 475, 476]

emotions = {
    "happy": {"emoji": "😊", "color": (0, 255, 0), "msg": "You look happy!"},
    "sad": {"emoji": "😢", "color": (255, 0, 0), "msg": "You seem sad."},
    "angry": {"emoji": "😠", "color": (0, 0, 255), "msg": "You look angry!"},
    "surprise": {"emoji": "😲", "color": (0, 255, 255), "msg": "You're surprised!"},
    "neutral": {"emoji": "😐", "color": (200, 200, 200), "msg": "You look neutral."},
    "fear": {"emoji": "😨", "color": (255, 140, 0), "msg": "You look scared."},
    "disgust": {"emoji": "🤢", "color": (138, 43, 226), "msg": "You're feeling disgusted."}
}

prev_emotion = ""
last_spoken_time = time.time()

def distance(p1, p2):
    return np.linalg.norm(np.array([p1.x, p1.y]) - np.array([p2.x, p2.y]))

def get_emotion(landmarks):
    top_lip = landmarks[13]
    bottom_lip = landmarks[14]
    left_mouth = landmarks[61]
    right_mouth = landmarks[291]
    left_eye_top = landmarks[159]
    left_eye_bottom = landmarks[145]
    right_eye_top = landmarks[386]
    right_eye_bottom = landmarks[374]
    iris_left = landmarks[468]

    face_width = distance(landmarks[234], landmarks[454])
    mouth_open = distance(top_lip, bottom_lip) / face_width
    mouth_stretch = distance(left_mouth, right_mouth) / face_width
    eye_open = (distance(left_eye_top, left_eye_bottom) + distance(right_eye_top, right_eye_bottom)) / (2 * face_width)
    eye_center_y = (left_eye_top.y + left_eye_bottom.y + right_eye_top.y + right_eye_bottom.y) / 4
    sad_offset = iris_left.y - eye_center_y

    if mouth_stretch > 0.40 and mouth_open < 0.06:
        return "happy"
    elif mouth_open >= 0.12:
        return "surprise"
    elif 0.06 < mouth_open < 0.12:
        return "fear"
    elif sad_offset > 0.01 and eye_open < 0.04:
        return "sad"
    elif mouth_open < 0.03 and eye_open < 0.08 and mouth_stretch < 0.38:
        return "disgust"
    elif eye_open > 0.096 and mouth_open < 0.06:
        return "angry"
    else:
        return "neutral"

def speak_emotion(msg):
    global last_spoken_time
    if time.time() - last_spoken_time > 2:
        threading.Thread(target=lambda: engine.say(msg) or engine.runAndWait()).start()
        last_spoken_time = time.time()

# Start webcam (640x480)
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    avatar_canvas = np.zeros_like(frame)

    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        face_landmarks = results.multi_face_landmarks[0]
        landmark_list = face_landmarks.landmark

        if frame_count % 5 == 0:  # Reduce emotion checks to every 5 frames
            emotion = get_emotion(landmark_list)

            if emotion != prev_emotion:
                speak_emotion(emotions[emotion]["msg"])
                prev_emotion = emotion
        else:
            emotion = prev_emotion

        color = emotions[emotion]["color"]
        emoji = emotions[emotion]["emoji"]

        # Draw avatar only on right
        drawing_utils.draw_landmarks(
            avatar_canvas, face_landmarks,
            mp_face_mesh.FACEMESH_TESSELATION, None,
            drawing_utils.DrawingSpec(color=color, thickness=1, circle_radius=1)
        )

        for idx in LEFT_IRIS + RIGHT_IRIS:
            pt = landmark_list[idx]
            cx, cy = int(pt.x * frame.shape[1]), int(pt.y * frame.shape[0])
            cv2.circle(avatar_canvas, (cx, cy), 2, (0, 255, 255), -1)

        cx, cy = int(landmark_list[1].x * frame.shape[1]), int(landmark_list[1].y * frame.shape[0])
        cv2.putText(avatar_canvas, f"{emotion.upper()} {emoji}", (cx - 40, cy - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    frame_count += 1

    # Combine webcam + avatar canvas
    combined = np.hstack((frame, avatar_canvas))
    cv2.imshow("Webcam | Avatar Emotion", combined)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


Exception in thread Thread-472 (<lambda>):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Deeksha\AppData\Local\Temp\ipykernel_31020\2326136763.py", line 74, in <lambda>
  File "C:\Users\Deeksha\AppData\Roaming\Python\Python312\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started


In [9]:
!pip install mediapipe opencv-python pyttsx3 numpy


Defaulting to user installation because normal site-packages is not writeable


In [17]:
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3
import threading
import time
import os

# TTS engine
engine = pyttsx3.init()
engine.setProperty('rate', 160)

# Load cartoon avatar images
emotion_images = {
    "happy": cv2.imread("avatar/happy.png"),
    "sad": cv2.imread("avatar/sad.png"),
    "angry": cv2.imread("avatar/angry.png"),
    "surprise": cv2.imread("avatar/surprise.png"),
    "fear": cv2.imread("avatar/fear.png"),
    "disgust": cv2.imread("avatar/disgust.png"),
    "neutral": cv2.imread("avatar/base.png")
}

# Resize all avatars to 480x480
for key in emotion_images:
    emotion_images[key] = cv2.resize(emotion_images[key], (480, 480))

# Mediapipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)

prev_emotion = ""
last_spoken_time = time.time()

def distance(p1, p2):
    return np.linalg.norm(np.array([p1.x, p1.y]) - np.array([p2.x, p2.y]))

def get_emotion(landmarks):
    top_lip = landmarks[13]
    bottom_lip = landmarks[14]
    left_mouth = landmarks[61]
    right_mouth = landmarks[291]
    left_eye_top = landmarks[159]
    left_eye_bottom = landmarks[145]
    right_eye_top = landmarks[386]
    right_eye_bottom = landmarks[374]
    iris_left = landmarks[468]

    face_width = distance(landmarks[234], landmarks[454])
    mouth_open = distance(top_lip, bottom_lip) / face_width
    mouth_stretch = distance(left_mouth, right_mouth) / face_width
    eye_open = (distance(left_eye_top, left_eye_bottom) + distance(right_eye_top, right_eye_bottom)) / (2 * face_width)
    eye_center_y = (left_eye_top.y + left_eye_bottom.y + right_eye_top.y + right_eye_bottom.y) / 4
    sad_offset = iris_left.y - eye_center_y

    if mouth_stretch > 0.40 and mouth_open < 0.06:
        return "happy"
    elif mouth_open >= 0.12:
        return "surprise"
    elif 0.06 < mouth_open < 0.12:
        return "fear"
    elif sad_offset > 0.01 and eye_open < 0.04:
        return "sad"
    elif mouth_open < 0.03 and eye_open < 0.08 and mouth_stretch < 0.38:
        return "disgust"
    elif eye_open > 0.096 and mouth_open < 0.06:
        return "angry"
    else:
        return "neutral"

def speak_emotion(msg):
    global last_spoken_time
    if time.time() - last_spoken_time > 2:
        threading.Thread(target=lambda: engine.say(msg) or engine.runAndWait()).start()
        last_spoken_time = time.time()

# Open webcam
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 480)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        face_landmarks = results.multi_face_landmarks[0]
        landmark_list = face_landmarks.landmark

        if frame_count % 5 == 0:
            emotion = get_emotion(landmark_list)
            if emotion != prev_emotion:
                speak_emotion({
                    "happy": "You look happy!",
                    "sad": "You seem sad.",
                    "angry": "You look angry!",
                    "surprise": "You're surprised!",
                    "neutral": "You look neutral.",
                    "fear": "You look scared.",
                    "disgust": "You're feeling disgusted."
                }[emotion])
                prev_emotion = emotion
        else:
            emotion = prev_emotion
    else:
        emotion = "neutral"

    # Get the cartoon avatar for detected emotion
    avatar = emotion_images.get(emotion, emotion_images["neutral"])
    combined = np.hstack((frame, avatar))
    cv2.imshow("Real Face | Avatar Emotion", combined)

    frame_count += 1

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [11]:
import os
print(os.listdir("avatar"))


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'avatar'

In [12]:
import os
print(os.listdir("avatar"))


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'avatar'

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3
import threading
import time
from queue import Queue

# Initialize TTS engine
engine = pyttsx3.init()
engine.setProperty('rate', 160)

# Queue and threading for speech
speech_queue = Queue()
stop_signal = threading.Event()

def speech_loop():
    while not stop_signal.is_set():
        if not speech_queue.empty():
            msg = speech_queue.get()
            engine.say(msg)
            engine.runAndWait()
        time.sleep(0.1)

def speak_emotion(msg):
    if speech_queue.empty():
        speech_queue.put(msg)

# Mediapipe face mesh setup
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)
drawing_utils = mp.solutions.drawing_utils

# Facial landmark sets
LIPS = mp_face_mesh.FACEMESH_LIPS
LEFT_IRIS = [468, 469, 470, 471]
RIGHT_IRIS = [473, 474, 475, 476]

emotions = {
    "happy": {"emoji": "😊", "color": (0, 255, 0), "msg": "You look happy!"},
    "sad": {"emoji": "😢", "color": (255, 0, 0), "msg": "You seem sad."},
    "angry": {"emoji": "😠", "color": (0, 0, 255), "msg": "You look angry!"},
    "surprise": {"emoji": "😲", "color": (0, 255, 255), "msg": "You're surprised!"},
    "neutral": {"emoji": "😐", "color": (200, 200, 200), "msg": "You look neutral."},
    "fear": {"emoji": "😨", "color": (255, 140, 0), "msg": "You look scared."},
    "disgust": {"emoji": "🤢", "color": (138, 43, 226), "msg": "You're feeling disgusted."}
}

def distance(p1, p2):
    return np.linalg.norm(np.array([p1.x, p1.y]) - np.array([p2.x, p2.y]))

def get_emotion(landmarks):
    top_lip = landmarks[13]
    bottom_lip = landmarks[14]
    left_mouth = landmarks[61]
    right_mouth = landmarks[291]
    left_eye_top = landmarks[159]
    left_eye_bottom = landmarks[145]
    right_eye_top = landmarks[386]
    right_eye_bottom = landmarks[374]
    iris_left = landmarks[468]

    face_width = distance(landmarks[234], landmarks[454])
    mouth_open = distance(top_lip, bottom_lip) / face_width
    mouth_stretch = distance(left_mouth, right_mouth) / face_width
    eye_open = (distance(left_eye_top, left_eye_bottom) + distance(right_eye_top, right_eye_bottom)) / (2 * face_width)
    eye_center_y = (left_eye_top.y + left_eye_bottom.y + right_eye_top.y + right_eye_bottom.y) / 4
    sad_offset = iris_left.y - eye_center_y

    if mouth_stretch > 0.40 and mouth_open < 0.06:
        return "happy"
    elif mouth_open >= 0.12:
        return "surprise"
    elif 0.06 < mouth_open < 0.12:
        return "fear"
    elif sad_offset > 0.01 and eye_open < 0.04:
        return "sad"
    elif mouth_open < 0.03 and eye_open < 0.08 and mouth_stretch < 0.38:
        return "disgust"
    elif eye_open > 0.096 and mouth_open < 0.06:
        return "angry"
    else:
        return "neutral"

# Start webcam
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

prev_emotion = ""
frame_count = 0

# Start speaker thread
speaker_thread = threading.Thread(target=speech_loop, daemon=True)
speaker_thread.start()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    avatar_canvas = np.zeros_like(frame)

    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        face_landmarks = results.multi_face_landmarks[0]
        landmark_list = face_landmarks.landmark

        if frame_count % 2 == 0:  # Faster detection
            emotion = get_emotion(landmark_list)

            if emotion != prev_emotion:
                prev_emotion = emotion
                speak_emotion(emotions[emotion]["msg"])
        else:
            emotion = prev_emotion

        color = emotions[emotion]["color"]
        emoji = emotions[emotion]["emoji"]

        # Draw face mesh on avatar canvas
        drawing_utils.draw_landmarks(
            avatar_canvas, face_landmarks,
            mp_face_mesh.FACEMESH_TESSELATION, None,
            drawing_utils.DrawingSpec(color=color, thickness=1, circle_radius=1)
        )

        # Draw iris points
        for idx in LEFT_IRIS + RIGHT_IRIS:
            pt = landmark_list[idx]
            cx, cy = int(pt.x * frame.shape[1]), int(pt.y * frame.shape[0])
            cv2.circle(avatar_canvas, (cx, cy), 2, (0, 255, 255), -1)

        # Show emotion text
        cx, cy = int(landmark_list[1].x * frame.shape[1]), int(landmark_list[1].y * frame.shape[0])
        cv2.putText(avatar_canvas, f"{emotion.upper()} {emoji}", (cx - 40, cy - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    frame_count += 1

    # Combine original + avatar canvas
    combined = np.hstack((frame, avatar_canvas))
    cv2.imshow("Webcam | Avatar Emotion", combined)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC to quit
        break

# Cleanup
stop_signal.set()
speaker_thread.join()
cap.release()
cv2.destroyAllWindows()


Exception in thread Thread-477 (speech_loop):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Deeksha\AppData\Local\Temp\ipykernel_31020\3817472991.py", line 22, in speech_loop
  File "C:\Users\Deeksha\AppData\Roaming\Python\Python312\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
